# Cytnx Tutorial 1 --iTEBD

In [172]:
import sys
sys.path.append("/usr/local/")
import cytnx
from cytnx import cytnx_extension as cyx

In [173]:
chi = 20
Hx = 1.0
CvgCrit = 1.0e-10
dt = 0.1

In [174]:
Sz = cytnx.zeros([2,2])
Sz[0,0] = 1
Sz[1,1] = -1

Sx = cytnx.zeros([2,2])
Sx[0,1] = Sx[1,0] = Hx

I = Sz.clone()
I[1,1] = 1
print('Sz = ', Sz)
print('Sx = ', Sx)


Sz =  
Total elem: 4
type  : Double (Float64)
cytnx device: CPU
Shape : (2,2)
[[1.00000e+00 0.00000e+00 ]
 [0.00000e+00 -1.00000e+00 ]]



Sx =  
Total elem: 4
type  : Double (Float64)
cytnx device: CPU
Shape : (2,2)
[[0.00000e+00 1.00000e+00 ]
 [1.00000e+00 0.00000e+00 ]]





In [175]:
## Build Evolution Operator
TFterm = cytnx.linalg.Kron(Sx,I) + cytnx.linalg.Kron(I,Sx)
ZZterm = cytnx.linalg.Kron(Sz,Sz)

H = TFterm + ZZterm
Htemp = H.clone()
H.reshape_(2,2,2,2)
print('H is Htemp : ',H is Htemp)
print('H = ', H)

H is Htemp :  False
H =  
Total elem: 16
type  : Double (Float64)
cytnx device: CPU
Shape : (2,2,2,2)
[[[[1.00000e+00 1.00000e+00 ]
   [1.00000e+00 0.00000e+00 ]]
  [[1.00000e+00 -1.00000e+00 ]
   [0.00000e+00 1.00000e+00 ]]]
 [[[1.00000e+00 0.00000e+00 ]
   [-1.00000e+00 1.00000e+00 ]]
  [[0.00000e+00 1.00000e+00 ]
   [1.00000e+00 1.00000e+00 ]]]]





# Question
- I don't understand the meaning of '2' in cyx.CyTensor(eH,2). Will it influence our calculation? Or it only matters when we want to print diagram? Or... doing Svd?

In [176]:
import numpy as np
from scipy import linalg as LA

eH = cytnx.linalg.ExpH(Htemp,-dt) ## or equivantly ExpH(-dt*H)
eTest = cytnx.linalg.ExpH(-dt*Htemp)
eH.reshape_(2,2,2,2)
eTest.reshape_(2,2,2,2)
print('LA.norm(eH-eTest) = ', LA.norm(eH-eTest)) 

#### Transform cytnx.Tensor() to cyx.CyTensor
eH = cyx.CyTensor(eH,2)
eH.print_diagram()
H = cyx.CyTensor(H,2)
H.print_diagram()

LA.norm(eH-eTest) =  5.812126222352033e-16
-----------------------
tensor Name : 
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 2  
           |             |     
     1 ____| 2         2 |____ 3  
           \             /     
            -------------      
-----------------------
tensor Name : 
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 2  
           |             |     
     1 ____| 2         2 |____ 3  
           \             /     
            -------------      


In [177]:
A = cyx.CyTensor([cyx.Bond(chi),cyx.Bond(2),cyx.Bond(chi)],rowrank=1,labels=[-1,0,-2]);
B = cyx.CyTensor(A.bonds(),rowrank=1,labels=[-3,1,-4]);
A.print_diagram()

-----------------------
tensor Name : 
tensor Rank : 3
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
    -1 ____| 20        2 |____ 0  
           |             |     
           |          20 |____ -2 
           \             /     
            -------------      


In [178]:
cytnx.random.Make_normal(B.get_block_(),0,0.2); ## this is temporary, will have more intuitive way to rand
cytnx.random.Make_normal(A.get_block_(),0,0.2); ## this is temporary, will have more intuitive way to randd

print(A[0][1][3].item())
print(A[0,1,3].item())

0.4432781552828508
0.4432781552828508


In [179]:
la = cyx.CyTensor([cyx.Bond(chi),cyx.Bond(chi)],rowrank=1,labels=[-2,-3],is_diag=True)
lb = cyx.CyTensor([cyx.Bond(chi),cyx.Bond(chi)],rowrank=1,labels=[-4,-5],is_diag=True)
la.put_block(cytnx.ones(chi));
lb.put_block(cytnx.ones(chi));
la.set_name('la')
lb.set_name('lb')
la.print_diagram()
lb.print_diagram()
print('la.shape = ',la.shape())
# print(la[0][1])

-----------------------
tensor Name : la
tensor Rank : 2
block_form  : false
is_diag     : True
on device   : cytnx device: CPU
            -------------      
           /             \     
    -2 ____| 20       20 |____ -3 
           \             /     
            -------------      
-----------------------
tensor Name : lb
tensor Rank : 2
block_form  : false
is_diag     : True
on device   : cytnx device: CPU
            -------------      
           /             \     
    -4 ____| 20       20 |____ -5 
           \             /     
            -------------      
la.shape =  [20, 20]


In [180]:
X = cyx.Contract(cyx.Contract(A,la),cyx.Contract(B,lb))
X.print_diagram()
Xtemp = X.clone()

-----------------------
tensor Name : 
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
    -1 ____| 20        2 |____ 0  
           |             |     
           |           2 |____ 1  
           |             |     
           |          20 |____ -5 
           \             /     
            -------------      


In [181]:
lb.set_label(idx=1,new_label=-1)
lb.print_diagram()

-----------------------
tensor Name : lb
tensor Rank : 2
block_form  : false
is_diag     : True
on device   : cytnx device: CPU
            -------------      
           /             \     
    -4 ____| 20       20 |____ -1 
           \             /     
            -------------      


In [184]:
X = cyx.Contract(lb,Xtemp)
X.print_diagram()
print(X[5][1][1][2].item())


-----------------------
tensor Name : 
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
    -4 ____| 20        2 |____ 0  
           |             |     
           |           2 |____ 1  
           |             |     
           |          20 |____ -5 
           \             /     
            -------------      
0.020827111458187685


In [185]:
Xt = X.clone()
Xt.set_name('Xt')
Xt.print_diagram()
XNorm = cyx.Contract(X,Xt).item()
print('XNorm = ', XNorm)
# XNorm.print_diagram()
H.set_name('H')
H.print_diagram()
X.set_name('X')
X.print_diagram()
XH = cyx.Contract(X,H)
XH.set_name('XH')
XH.print_diagram()
XH.set_labels([-4,-5,0,1])
XH.print_diagram()
XHX = cyx.Contract(Xt,XH).item() ## rank-0
print('XHX = ', XHX)
E = XHX/XNorm
print('E = ', E)

-----------------------
tensor Name : Xt
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
    -4 ____| 20        2 |____ 0  
           |             |     
           |           2 |____ 1  
           |             |     
           |          20 |____ -5 
           \             /     
            -------------      
XNorm =  57.30980992642162
-----------------------
tensor Name : H
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
     0 ____| 2         2 |____ 2  
           |             |     
     1 ____| 2         2 |____ 3  
           \             /     
            -------------      
-----------------------
tensor Name : X
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \   

In [186]:
XeH = cyx.Contract(X,eH)
XeH.print_diagram()
XeH.permute_([-4,2,3,-5], by_label=True)
XeH.print_diagram()
XeH.set_Rowrank(2)

XeH.print_diagram()

-----------------------
tensor Name : 
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
    -4 ____| 20       20 |____ -5 
           |             |     
           |           2 |____ 2  
           |             |     
           |           2 |____ 3  
           \             /     
            -------------      
-----------------------
tensor Name : 
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
    -4 ____| 20        2 |____ 2  
           |             |     
           |           2 |____ 3  
           |             |     
           |          20 |____ -5 
           \             /     
            -------------      
-----------------------
tensor Name : 
tensor Rank : 4
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------

In [187]:
la,A,B = cyx.xlinalg.Svd_truncate(XeH,chi)
la.print_diagram()
A.print_diagram()
B.print_diagram()
testNorm = cytnx.linalg.Norm(la.get_block())
print('testNorm = ', testNorm)
Norm = cytnx.linalg.Norm(la.get_block_()).item()
print('Norm = ', Norm)

-----------------------
tensor Name : 
tensor Rank : 2
block_form  : false
is_diag     : True
on device   : cytnx device: CPU
            -------------      
           /             \     
    -6 ____| 20       20 |____ -7 
           \             /     
            -------------      
-----------------------
tensor Name : 
tensor Rank : 3
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
    -4 ____| 20       20 |____ -6 
           |             |     
     2 ____| 2           |        
           \             /     
            -------------      
-----------------------
tensor Name : 
tensor Rank : 3
block_form  : false
is_diag     : False
on device   : cytnx device: CPU
            -------------      
           /             \     
    -7 ____| 20        2 |____ 3  
           |             |     
           |          20 |____ -5 
           \             /     
            -------------    

# Calculation Time

In [188]:
Elast = 0
for i in range(10000):

    A.set_labels([-1,0,-2])
    B.set_labels([-3,1,-4])
    la.set_labels([-2,-3])
    lb.set_labels([-4,-5])

    ## contract all
    X = cyx.Contract(cyx.Contract(A,la),cyx.Contract(B,lb))
    #X.print_diagram()
    lb.set_label(idx=1,new_label=-1)
    X = cyx.Contract(lb,X)

    ## X =
    #           (0)  (1)
    #            |    |     
    #  (-4) --lb-A-la-B-lb-- (-5) 
    #
    #X.print_diagram()

    Xt = X.clone()

    ## calculate norm and energy for this step
    # Note that X,Xt contract will result a rank-0 tensor, which can use item() toget element
    XNorm = cyx.Contract(X,Xt).item()
    XH = cyx.Contract(X,H)
    XH.set_labels([-4,-5,0,1])
    XHX = cyx.Contract(Xt,XH).item() ## rank-0
    E = XHX/XNorm

    ## check if converged.
    if(np.abs(E-Elast) < CvgCrit):
        print("[Converged!]")
        break
    print("Step: %d Enr: %5.8f"%(i,Elast))
    Elast = E

    ## Time evolution the MPS
    XeH = cyx.Contract(X,eH)
    XeH.permute_([-4,2,3,-5],by_label=True)
    #XeH.print_diagram()
    
    ## Do Svd + truncate
    ## 
    #        (2)   (3)                   (2)                                    (3)
    #         |     |          =>         |         +   (-6)--s--(-7)  +         |
    #  (-4) --= XeH =-- (-5)        (-4)--U--(-6)                          (-7)--Vt--(-5)
    #

    XeH.set_Rowrank(2)
    la,A,B = cyx.xlinalg.Svd_truncate(XeH,chi)
    Norm = cytnx.linalg.Norm(la.get_block_()).item()
    la *= 1./Norm
    #A.print_diagram()
    #la.print_diagram()
    #B.print_diagram()
         

    # de-contract the lb tensor , so it returns to 
    #             
    #            |     |     
    #       --lb-A'-la-B'-lb-- 
    #
    # again, but A' and B' are updated 
    A.set_labels([-1,0,-2]); A.set_Rowrank(1);
    B.set_labels([-3,1,-4]); B.set_Rowrank(1);

    #A.print_diagram()
    #B.print_diagram()

    lb_inv = 1./lb
    A = cyx.Contract(lb_inv,A)
    B = cyx.Contract(B,lb_inv)

    #A.print_diagram()
    #B.print_diagram()

    # translation symmetry, exchange A and B site
    A,B = B,A
    la,lb = lb,la



Step: 0 Enr: 0.00000000
Step: 1 Enr: -0.63970991
Step: 2 Enr: -0.91103315
Step: 3 Enr: -1.51911087
Step: 4 Enr: -1.68930925
Step: 5 Enr: -1.94275371
Step: 6 Enr: -1.99953828
Step: 7 Enr: -2.06312950
Step: 8 Enr: -2.05853529
Step: 9 Enr: -2.07727335
Step: 10 Enr: -2.07276773
Step: 11 Enr: -2.07929675
Step: 12 Enr: -2.07628995
Step: 13 Enr: -2.07902491
Step: 14 Enr: -2.07767960
Step: 15 Enr: -2.07882565
Step: 16 Enr: -2.07817720
Step: 17 Enr: -2.07865937
Step: 18 Enr: -2.07837360
Step: 19 Enr: -2.07859123
Step: 20 Enr: -2.07846589
Step: 21 Enr: -2.07856253
Step: 22 Enr: -2.07850758
Step: 23 Enr: -2.07855035
Step: 24 Enr: -2.07852619
Step: 25 Enr: -2.07854542
Step: 26 Enr: -2.07853509
Step: 27 Enr: -2.07854395
Step: 28 Enr: -2.07853962
Step: 29 Enr: -2.07854373
Step: 30 Enr: -2.07854196
Step: 31 Enr: -2.07854389
Step: 32 Enr: -2.07854321
Step: 33 Enr: -2.07854416
Step: 34 Enr: -2.07854393
Step: 35 Enr: -2.07854442
Step: 36 Enr: -2.07854437
Step: 37 Enr: -2.07854462
Step: 38 Enr: -2.078544